In [1]:
!pip install h5py
!pip install googledrivedownloader
!pip install imageio

^C


In [14]:
from path import Path
import h5py
import imageio
import os
import random

In [15]:
from google_drive_downloader import GoogleDriveDownloader as gdd

## Download the data

In [21]:
DATASET = "partial"

if DATASET == "full":
    print("the full dataset is 6.0G, this download may take a while...")
    images_download_id = "1Ka0XfEMiwgCYPdTI-vv6eUElOBnKFKQ2"
    lagels_download_id = "1269yhu3pZDP8UYFQs-NYs3FPwuK-nGSG"
elif DATASET == "partial":
    images_download_id = "1hgshYGWK8V-eGRy8LToWJJgDU_rXWVJ3"
    lagels_download_id = "1bH8ZRbhSVAhScTS0p9-ZzGnX91cHT3uO"
    
path = Path("./histology")
x_h5_gz = path/"x.h5.gz"
y_h5_gz = path/"y.h5.gz"
    
# download the images
gdd.download_file_from_google_drive(file_id=images_download_id,
                                    dest_path=x_h5_gz, 
                                    overwrite=True)
# download the labels
gdd.download_file_from_google_drive(file_id=lagels_download_id,
                                    dest_path=y_h5_gz,
                                    overwrite=True)
# uncompress the images
!gunzip {x_h5_gz}
# uncompress the labels
!gunzip {y_h5_gz}

gzip: ./histology/x.h5.gz: No such file or directory
gzip: ./histology/y.h5.gz: No such file or directory


## Extract the images

In [16]:
path=Path("./histology")

h5_file_x = h5py.File(path/"x.h5").get('x')
h5_file_y = h5py.File(path/"y.h5").get('y')

tumour_path = path/"data/tumour"
normal_path = path/"data/normal"

os.makedirs(normal_path, exist_ok=True)
os.makedirs(tumour_path, exist_ok=True)

images_to_extract = 20000

counter_tumour = 0
counter_normal = 0
for x, y in zip(h5_file_x, h5_file_y):
    if int(y[0,0,0]):
        imageio.imsave(tumour_path/str(counter_tumour) + ".png", x)
        counter_tumour += 1
    else:
        imageio.imsave(normal_path/str(counter_normal) + ".png", x)
        counter_normal += 1
    if counter_normal + counter_tumour == images_to_extract:
        break

TypeError: zip argument #1 must support iteration